In [1]:
import codecs, os, pickle, csv
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [2]:
os.chdir('../data/scrape/scraped/')
m = codecs.open('master.csv', 'r', encoding='utf-8', errors='ignore')
master = pd.read_csv(m)

In [3]:
train = master.sample(frac=0.9,random_state=200)
test = master.drop(train.index)

In [3]:
masterTuples = [list(x) for x in master.values]

In [4]:
trainTuples = [list(x) for x in train.values]
testTuples = [list(x) for x in train.values]

In [4]:
def getStopWordList(stopWordListFileName):
    stopwords = []
    
    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords

stopWords = []
st = open('../../stopwords.txt', 'r')
stopWords = getStopWordList('../../stopwords.txt')

In [5]:
vectorizer = CountVectorizer(stop_words=stopWords)
master_features = vectorizer.fit_transform(master['quote'].values.astype(str))

#train_features = vectorizer.fit_transform(train['quote'].values.astype(str))
#test_features = vectorizer.transform(test['quote'].values.astype(str))

nb = MultinomialNB()
nb.fit(master_features, master['atmosphere'].values.astype(str))

#nb.fit(train_features, train['atmosphere'].values.astype(str))
#predictions = nb.predict(test_features)

#actual = test['atmosphere'].values.astype(str)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [7]:
acc = metrics.accuracy_score(actual, predictions)
print("Accuracy: ", acc)

Accuracy:  0.535145726182


In [10]:
testSent = ['she died later that night', 'i kicked it', 'the spa was so relaxing']
testSentFeatures = vectorizer.transform(testSent)
nb.predict(testSentFeatures)

array(['sadness', 'fight', 'peaceful'], 
      dtype='<U11')

In [6]:
savedClassifier = open('savedNBClassifier2.pkl', 'wb')
pickle.dump(nb, savedClassifier)
pickle.dump(vectorizer, savedClassifier)
savedClassifier.close()

In [14]:
savedClassifier = open('savedNBClassifier.pkl', 'rb')
classer = pickle.load(savedClassifier)
vectorer = pickle.load(savedClassifier)
savedClassifier.close()

In [15]:
testSent = ['she died later that night', 'i kicked it', 'the spa was so relaxing']
testSentFeatures = vectorer.transform(testSent)
classer.predict(testSentFeatures)

array(['sadness', 'fight', 'peaceful'], 
      dtype='<U11')